# Convert Osu to Beet Saber

## 1. Convert .osz to .osu

In [1]:
# find install dir
import os
import zipfile

In [2]:
osz_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/'
unzip_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/charts/'

In [3]:
# find all osu zips
oszs = [osz_path+x for x in os.listdir(osz_path) if x.endswith('osz')]
oszs

['/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/killing-my-love.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/ETA.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/stupid-horse.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/duvet.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/running-man.osz']

In [4]:
def retain_file(filepath):
  """Returns if file should be kept
  At time of writing, keeps osu metadata files and mp3 files
  (wav files are used only for hitsounds (irrelevant))"""
  if not (filepath.endswith('osu') or filepath.endswith('mp3')):
    return False
  return True

In [5]:
def keep_file(filepath):
  """
  Tells which files should be kept
  """
  checks = ['easy' in filepath.lower(),
            'normal' in filepath.lower(),
            'hard' in filepath.lower(),
            'insane' in filepath.lower(),
            'extra' in filepath.lower(),
            'expert' in filepath.lower(),
            'audio.mp3' in filepath,
            ]
  # !impt: taiko diffs are like so: kantan, futsuu, muzukashii, oni
  
  return ((True in checks) and ('.wav' not in filepath))

In [6]:
# unzip them. how?
# # make dir per song
# # retain metadata and audio
os.makedirs(unzip_path, exist_ok=True)
for osz in oszs:
  songname = osz.split('/')[-1].split('.')[0]
  save_path = unzip_path+songname+'/'
  with zipfile.ZipFile(osz, 'r') as zip_ref:
    zip_ref.extractall(save_path)

# delete unneeded files:
# # subdirs is minimal directory paths (e.g. killing-my-love, duvet w.o. abspath)
subdirs = [entry for entry in os.listdir(unzip_path) if os.path.isdir(os.path.join(unzip_path, entry))]
for subdir in subdirs:
  dirpath = os.path.join(unzip_path,subdir)
  for file in os.listdir(dirpath):
    if not keep_file(file):
      os.remove(os.path.join(dirpath,file))

## 2. Extract Hit Objects from .osu

In [7]:
# wiki is extremely well documented
# # https://osu.ppy.sh/wiki/en/Client/File_formats/osu_%28file_format%29#type:~:text=Slider%20border%20colour-,Hit%20objects,-Hit%20object%20syntax

In [8]:
example_song = os.path.join(unzip_path,'lovenote')

In [9]:
example_osu_dir = os.listdir(example_song)
example_osu_dir

['audio.mp3']

In [10]:
#example_osu #= example_osu_dir[2]

In [11]:
def classify_object(flag):
  """
  Derive object type from osu's binary flags
  
  Simplied representation:
  - 'circle'
  - 'slider'
  - 'spinner'
  
  """
  # 0 bit indiates slider
  circle = (flag & 1) != 0 # check neq 0 not eq 1 b/c bit shift
  slider = (flag & 1<<1) != 0
  spinner = (flag & 1<<3) != 0
  
  if circle+slider+spinner>1:
    print("Whaaat")
  if circle: 
    return 'circle'
  if slider: 
    return 'slider'
  if spinner: 
    return 'spinner'
  print(f"Something is wrong in object classification. Recieved flag: {flag}")
  return None

A quick note about taiko. Besides naming convention for difficulties, taiko and osu!std objects are stored identically. hype

In [31]:
def extract_hit_objects(osu_path):
  """
  Find type information and timing data for all hit objects contained in an .osu file
  """
  start_str = '[HitObjects]'
  #csv_fields = 'x,y,time,type,hitSound,objectParams,hitSample'.split(',') # from the wiki
  note_types = []
  note_times = []
  with open(osu_path, 'r') as map_file:
    content = map_file.read()
    #print(content.find(start_str))
    # see csv_fields comment for how hit objects are parameterized
    hit_objects = content[content.find(start_str)+len(start_str)+1:].split('\n') # extract hit objects
    #print(len(hit_objects))
    # for now, just time and circle/slider class
    for hit_object in hit_objects:
      if len(hit_object)>5: # there will always be at least 6 params
        hit_object_fields = hit_object.split(',')
        #print(hit_object_fields)
        ms, note_type = hit_object_fields[2],hit_object_fields[3]
        note_types.append(classify_object(int(note_type)))
        note_times.append(int(ms))
  return list(zip(note_types,note_times))
  #return pd.read_csv(content)#,usecols=csv_fields)

In [41]:
song = "/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/charts/killing-my-love/LESLIE PARRISH - KILLING MY LOVE (Cut Ver.) (Pincus) [Pepekcz's Hard Ryosuke's RX-7 FC].osu"
hitobjs = extract_hit_objects(song)

In [42]:
# type conversion
print(len(hitobjs))
for i in hitobjs:
  print(i)


328
('slider', 1675)
('slider', 2058)
('circle', 2345)
('slider', 2537)
('slider', 2824)
('slider', 3207)
('slider', 3590)
('circle', 3877)
('slider', 4068)
('circle', 4355)
('circle', 4547)
('circle', 4738)
('circle', 6078)
('circle', 6174)
('slider', 6270)
('circle', 6844)
('circle', 7036)
('circle', 7227)
('slider', 7419)
('circle', 7993)
('circle', 8089)
('slider', 8184)
('slider', 8567)
('circle', 8950)
('slider', 9333)
('circle', 9907)
('circle', 10099)
('circle', 10290)
('slider', 10482)
('circle', 11056)
('circle', 11152)
('slider', 11248)
('slider', 11630)
('circle', 12013)
('slider', 12396)
('circle', 12971)
('circle', 13162)
('slider', 13353)
('circle', 13928)
('circle', 14119)
('circle', 14215)
('slider', 14311)
('circle', 14694)
('slider', 14885)
('slider', 15459)
('circle', 16034)
('circle', 16225)
('slider', 16417)
('circle', 16991)
('slider', 17182)
('circle', 17565)
('circle', 17757)
('slider', 17948)
('circle', 18523)
('circle', 18716)
('circle', 18908)
('circle', 191